# 요구사항 1

## Section 1: Imports and Dataset Class Definitions

In [12]:
# Section 1: Imports and Dataset Class Definitions
import os # os: 파일 및 디렉토리 경로 조작에 사용
import torch # torch: PyTorch 라이브러리, 딥러닝을 위한 주요 패키지
import pandas as pd # pandas: 데이터프레임 구조를 사용하여 데이터 처리 및 분석
from torch import nn, optim # nn, optim: PyTorch의 신경망 계층 및 옵티마이저 정의
from torch.utils.data import Dataset, DataLoader, random_split # Dataset, DataLoader, random_split: PyTorch에서 데이터를 로드하고, 배치로 나누고, 훈련/검증 데이터셋을 분할하는 데 사용
from datetime import datetime # datetime: 현재 시간 및 날짜 처리
import wandb # wandb: 학습 과정을 추적하고 시각화하는 WandB 라이브러리
import argparse # argparse: 명령줄 인자 파싱
from sklearn.preprocessing import StandardScaler # StandardScaler: scikit-learn에서 제공하는 표준화 도구로, 데이터의 평균을 0으로, 분산을 1로 조정
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score # accuracy_score, precision_score, recall_score, f1_score: scikit-learn에서 제공하는 성능 평가 메트릭


## Section 2: # TitanicDataset Class for Train and Validation Data

In [13]:
# TitanicDataset Class for Train and Validation Data

class TitanicDataset(Dataset):
    def __init__(self, X, y):
        # 초기화 메서드: 입력 데이터(X)와 타겟 데이터(y)를 텐서로 변환하여 저장
        self.X = torch.FloatTensor(X)  # 입력 데이터를 FloatTensor로 변환
        self.y = torch.LongTensor(y)   # 타겟 데이터를 LongTensor로 변환 (분류 작업에 적합)
        
    def __len__(self):
        # 데이터셋의 길이를 반환하는 메서드
        return len(self.X)
    
    def __getitem__(self, idx):
        # 인덱스(idx)에 해당하는 데이터 포인트를 반환하는 메서드
        return {'input': self.X[idx], 'target': self.y[idx]}
    
    def __str__(self):
        # 데이터셋의 정보를 문자열로 반환하는 메서드
        return f"Data Size: {len(self.X)}, Input Shape: {self.X.shape}, Target Shape: {self.y.shape}"


## Section 3: TitanicTestDataset

In [14]:
class TitanicTestDataset(Dataset):
    def __init__(self, X, passenger_ids):
        # 초기화 메서드: 입력 데이터(X)를 FloatTensor로 변환하고, PassengerId를 저장
        self.X = torch.FloatTensor(X)  # 입력 데이터를 FloatTensor로 변환
        self.passenger_ids = passenger_ids  # PassengerId 리스트 저장
        
    def __len__(self):
        # 데이터셋의 길이를 반환하는 메서드
        return len(self.X)
    
    def __getitem__(self, idx):
        # 인덱스(idx)에 해당하는 데이터 포인트를 반환하는 메서드
        feature = self.X[idx]  # 입력 데이터
        passenger_id = self.passenger_ids[idx]  # 해당 인덱스의 PassengerId
        return {'input': feature, 'PassengerId': passenger_id}  # 입력 특징과 PassengerId 반환

## Section 4: get_preprocessed_dataset_1-6

In [15]:
def get_preprocessed_dataset_1(df):
    # 성별 인코딩 (male=0, female=1)
    df['Sex'] = df['Sex'].map({'male': 0, 'female': 1})  # 성별을 숫자로 인코딩하여 모델이 처리할 수 있도록 변환
    return df

def get_preprocessed_dataset_2(df):
    # 결측값 처리 (Age의 결측값을 평균으로 채움)
    df['Age'] = df['Age'].fillna(df['Age'].mean())  # Age의 결측값을 평균으로 대체
    return df

def get_preprocessed_dataset_3(df):
    # 결측값 처리 (Fare의 결측값을 평균으로 채움)
    df['Fare'] = df['Fare'].fillna(df['Fare'].mean())  # Fare의 결측값을 평균으로 대체
    return df

def get_preprocessed_dataset_4(df):
    # 불필요한 열 제거, 단 PassengerId는 제거하지 않음
    columns_to_drop = ['Name', 'Ticket', 'Cabin', 'Embarked']  # 분석에 불필요한 열 목록
    df = df.drop(columns=columns_to_drop, axis=1, errors='ignore')  # 지정된 열을 데이터프레임에서 제거
    return df

def get_preprocessed_dataset_5(df):
    # Pclass를 범주형에서 연속형으로 처리
    df['Pclass'] = df['Pclass'].astype(float)  # 클래스 열을 float로 변환하여 연속형 변수로 사용
    return df

def get_preprocessed_dataset_6(df):
    # 정규화 (필요한 경우)
    scaler = StandardScaler()  # 데이터의 분포를 표준화하여 평균을 0, 표준편차를 1로 조정
    features = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare']  # 정규화할 특징 목록
    df[features] = scaler.fit_transform(df[features])  # 특징들을 정규화하여 모델의 수렴 속도를 높임
    return df


## Section 5 : get_preprocessed_dataset

In [16]:
# Section 2: Data Preprocessing
def get_preprocessed_dataset():
    # 현재 작업 디렉토리 경로를 기준으로 데이터 파일 경로 설정
    CURRENT_FILE_PATH = os.getcwd()
    train_data_path = os.path.join(CURRENT_FILE_PATH, "train.csv")
    test_data_path = os.path.join(CURRENT_FILE_PATH, "test.csv")
    
    # 훈련 데이터와 테스트 데이터를 각각 로드하여 DataFrame 형태로 저장
    train_df = pd.read_csv(train_data_path)
    test_df = pd.read_csv(test_data_path)
    
    # 훈련 및 테스트 데이터를 하나의 DataFrame으로 병합 (전처리를 한 번에 수행하기 위함)
    all_df = pd.concat([train_df, test_df], sort=False)
    
    # 전처리 함수들을 순차적으로 호출하여 데이터셋 전처리 수행
    all_df = get_preprocessed_dataset_1(all_df)  # 성별 인코딩
    all_df = get_preprocessed_dataset_2(all_df)  # Age 결측값 처리
    all_df = get_preprocessed_dataset_3(all_df)  # Fare 결측값 처리
    all_df = get_preprocessed_dataset_4(all_df)  # 불필요한 열 제거
    all_df = get_preprocessed_dataset_5(all_df)  # Pclass 변환
    all_df = get_preprocessed_dataset_6(all_df)  # 정규화
    
    return all_df  # 전처리된 데이터프레임 반환


## Section 6 : Model Definition

In [17]:
# Section 3: Model Definition version 2
class MyModel(nn.Module):
    def __init__(self, n_input, n_output):
        super().__init__()
        # 신경망 계층 정의: ELU 활성화 함수와 Dropout을 통해 과적합 방지
        self.model = nn.Sequential(
            nn.Linear(n_input, 256),  # 첫 번째 은닉층, 입력 크기에서 256개의 뉴런으로 연결
            nn.ELU(),                 # ELU 활성화 함수, 비선형성을 추가해 복잡한 패턴 학습
           # nn.Dropout(0.3),          # 30%의 뉴런을 무작위로 비활성화하여 과적합 방지
            nn.Linear(256, 64),       # 두 번째 은닉층, 64개의 뉴런으로 연결
            nn.ELU(),                 # ELU 활성화 함수
           # nn.Dropout(0.3),          # 30%의 Dropout 추가
            nn.Linear(64, n_output),  # 출력층, n_output 클래스 수로 연결
            nn.Sigmoid()          # 이진 분류를 위해 Sigmoid 활성화 추가
        )

    def forward(self, x):
        # 순전파 정의: 입력 x를 신경망 계층에 통과시켜 결과를 반환
        return self.model(x)


In [18]:
# Section 4: Training and Evaluation Loops_v2_wandB에 추가하기

def training_loop(model, optimizer, train_data_loader, validation_data_loader):
    n_epochs = wandb.config.epochs  # WandB 설정에서 에포크 수 가져오기
    loss_fn = nn.CrossEntropyLoss()  # 다중 클래스 분류 손실 함수 정의

    for epoch in range(1, n_epochs + 1):
        # Training Loop
        model.train()  # 모델을 학습 모드로 설정
        train_loss = 0.0  # 훈련 손실 초기화
        for batch in train_data_loader:
            inputs, targets = batch['input'], batch['target']  # 입력과 타겟 분리
            outputs = model(inputs)  # 모델을 통한 예측
            loss = loss_fn(outputs, targets)  # 손실 계산
            train_loss += loss.item()  # 배치 손실 합산
            
            optimizer.zero_grad()  # 기울기 초기화
            loss.backward()  # 역전파로 기울기 계산
            optimizer.step()  # 가중치 업데이트

        train_loss /= len(train_data_loader)  # 에포크별 평균 훈련 손실 계산

        # Validation Loss 계산
        model.eval()  # 모델을 평가 모드로 설정
        validation_loss = 0.0  # 검증 손실 초기화
        with torch.no_grad():  # 검증 시 기울기 계산 방지
            for batch in validation_data_loader:
                inputs, targets = batch['input'], batch['target']
                outputs = model(inputs)
                loss = loss_fn(outputs, targets)
                validation_loss += loss.item()

        validation_loss /= len(validation_data_loader)  # 에포크별 평균 검증 손실 계산

        # WandB에 학습 및 검증 손실 기록
        wandb.log({"Training Loss": train_loss, "Validation Loss": validation_loss, "Epoch": epoch})

        # 에포크별 훈련 및 검증 손실 출력
        print(f"Epoch {epoch} - Training Loss: {train_loss:.4f}, Validation Loss: {validation_loss:.4f}")


## Section 7 : predict_test

In [19]:
# predict_test version 5
# Section 5: Model Testing and Prediction

import pandas as pd

# Model Testing and Prediction
def predict_test(model, test_data_loader, output_path="test_predictions_ELU_testing_almost_final.csv"):
    model.eval()  # 모델을 평가 모드로 설정
    predictions = []  # 예측 결과를 저장할 리스트
    passenger_ids = []  # PassengerId 저장을 위한 리스트

    # 예측 수행
    with torch.no_grad():  # 평가 시에는 기울기 계산 비활성화
        for batch in test_data_loader:
            inputs = batch['input']  # 입력 데이터
            passenger_ids_batch = batch['PassengerId']  # PassengerId 가져오기
            outputs = model(inputs)  # 모델을 통해 예측 수행
            predicted_labels = torch.argmax(outputs, dim=1)  # 예측 확률 중 가장 높은 값을 가진 클래스로 변환
            
            # 예측 값과 PassengerId 저장
            predictions.extend(predicted_labels.cpu().numpy())  # Tensor에서 numpy로 변환하여 리스트에 추가
            passenger_ids.extend(passenger_ids_batch.cpu().numpy())  # Tensor에서 numpy로 변환하여 리스트에 추가

    # 예측 결과를 DataFrame으로 변환
    preds_df = pd.DataFrame({
        "PassengerId": passenger_ids,  # PassengerId 열
        "Survived": predictions        # 예측된 Survived 값
    })
    
    # DataFrame을 CSV 파일로 저장
    preds_df.to_csv(output_path, index=False)  # 지정된 경로에 CSV 파일로 저장
    print(f"Test predictions saved to {os.path.abspath(output_path)}")  # 파일의 절대 경로 출력

    return predictions  # 예측 값 리스트 반환


## Section 8 : main function

In [20]:
# version 3
def main(args):
    # WandB 프로젝트 초기화 및 설정 로깅
    wandb.init(project="titanic_model_training", config={
        "epochs": args.epochs,
        "batch_size": args.batch_size,
        "learning_rate": args.learning_rate
    })

    # 전처리된 데이터셋 로드
    all_df = get_preprocessed_dataset()
    
    # 훈련, 검증, 테스트 데이터셋 분리
    train_df = all_df[all_df['Survived'].notna()]  # 생존 데이터(Survived) 존재 행을 훈련용으로 사용
    test_df = all_df[all_df['Survived'].isna()]    # 생존 데이터가 없는 행을 테스트용으로 사용
    
    # 특징(X)와 타겟(y) 분리
    X_train = train_df.drop(columns=['Survived']).values  # 타겟 열 제거하여 입력 데이터만 남김
    y_train = train_df['Survived'].values                 # 타겟 값만 분리
    X_test = test_df.drop(columns=['Survived']).values    # 테스트 데이터에서 타겟 열 제거
    passenger_ids_test = test_df['PassengerId'].values    # 테스트 데이터에서 PassengerId 가져오기

    # Dataset 생성
    full_train_dataset = TitanicDataset(X_train, y_train)
    
    # 훈련/검증 데이터 분리 (80% 훈련, 20% 검증)
    train_size = int(0.8 * len(full_train_dataset))  # 전체 훈련 데이터의 80%
    val_size = len(full_train_dataset) - train_size  # 나머지 20%를 검증에 사용
    train_dataset, validation_dataset = random_split(full_train_dataset, [train_size, val_size])
    
    # 테스트 데이터셋에 PassengerId 포함
    test_dataset = TitanicTestDataset(X_test, passenger_ids_test)

    # DataLoader 생성
    train_data_loader = DataLoader(train_dataset, batch_size=args.batch_size, shuffle=True)       # 훈련 데이터 로더
    validation_data_loader = DataLoader(validation_dataset, batch_size=args.batch_size, shuffle=True)  # 검증 데이터 로더
    test_data_loader = DataLoader(test_dataset, batch_size=len(test_dataset))                     # 테스트 데이터 로더

    # 모델 및 옵티마이저 초기화
    model = MyModel(n_input=X_train.shape[1], n_output=2)  # 입력 피처 수와 이진 분류에 맞춰 모델 정의
    optimizer = optim.Adam(model.parameters(), lr=args.learning_rate)  # Adam 옵티마이저로 학습 설정

    # 훈련 루프 실행
    training_loop(model, optimizer, train_data_loader, validation_data_loader)

    # 테스트 데이터에 대한 예측 수행 및 결과 출력
    test_predictions = predict_test(model, test_data_loader)
    print(f"Test predictions: {test_predictions}")

    wandb.finish()  # WandB 세션 종료


## Section 9: Argument Parsing and Main Execution

In [11]:
if __name__ == "__main__":
    # argparse.Namespace로 인자를 직접 설정합니다.
    args = argparse.Namespace(
        train_csv_path="train.csv",                # 실제 훈련 데이터 경로
        test_csv_path="test.csv",                  # 실제 테스트 데이터 경로
        output_path="test_predictions_ELU_testing_almost_final.csv",    # 테스트 예측 결과 파일 경로
        wandb=True,                                # WandB 사용 여부 (True로 설정하면 WandB 사용)
        batch_size=64,                             # 배치 사이즈
        epochs=300,                                # 학습 에폭 수
        learning_rate=0.001,                       # 학습률
        hidden_units=[128, 64]                     # 은닉층 크기 리스트
    )

    # main 함수 호출
    main(args)

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: -ddj127 (-ddj127-korea-university-of-technology-and-education). Use `wandb login --relogin` to force relogin


Epoch 1 - Training Loss: 0.7102, Validation Loss: 0.6849
Epoch 2 - Training Loss: 0.6784, Validation Loss: 0.6863
Epoch 3 - Training Loss: 0.6774, Validation Loss: 0.6910
Epoch 4 - Training Loss: 0.6963, Validation Loss: 0.6867
Epoch 5 - Training Loss: 0.6863, Validation Loss: 0.6970
Epoch 6 - Training Loss: 0.6951, Validation Loss: 0.6859
Epoch 7 - Training Loss: 0.7032, Validation Loss: 0.6892
Epoch 8 - Training Loss: 0.7209, Validation Loss: 0.6838
Epoch 9 - Training Loss: 0.6866, Validation Loss: 0.6849
Epoch 10 - Training Loss: 0.6952, Validation Loss: 0.6868
Epoch 11 - Training Loss: 0.6932, Validation Loss: 0.6818
Epoch 12 - Training Loss: 0.6669, Validation Loss: 0.6871
Epoch 13 - Training Loss: 0.6827, Validation Loss: 0.6829
Epoch 14 - Training Loss: 0.6933, Validation Loss: 0.6858
Epoch 15 - Training Loss: 0.6837, Validation Loss: 0.6885
Epoch 16 - Training Loss: 0.7021, Validation Loss: 0.6831
Epoch 17 - Training Loss: 0.6994, Validation Loss: 0.6838
Epoch 18 - Training Los

Epoch,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▅▅▅▅▅▅▅▅▆▆▆▇▇▇▇▇████
Training Loss,▅▄▄▇▃▅▃▃▄▄▃▅▃▇▄▅▅▇▄▃▅▂▆▃▃▅▅▃▁▂▃▃█▄▅▅▂▁▄▃
Validation Loss,▁▁▁▁▁▁▁▁▁▁▁██▁▁▁▁▁▁▁▁▁▁▁▁▁▁██▁▂▁▁▁▁▁▁▁▁▁
Epoch,300
Training Loss,0.66222
Validation Loss,0.6901


## 에러 발생

발견 - 하루 10개 이상 제출을 못함 

![competition78](./Per_day_allow.png)

# 요구사항2

기존 코드에서 수정하여 특정 데이터셋에 맞게 고치기

하단 코드의 경우 .py파일 기준으로 작성됨

python verseion : 3.1x\
os : windows11\
개발환경 : pycharm

![Rellu](./Rellu.png)

![PREU](./PREU.png)

![liki_Relu](./liki_Relu.png)

![ELU](./ELU.png)

![ReLU](./ReLU.png)

![validationLoss](./validationLoss.png)

address : https://wandb.ai/-ddj127-korea-university-of-technology-and-education/titanic_model_training

In [ ]:
import torch
from torch import nn, optim
from torch.utils.data import Dataset, random_split, DataLoader
from datetime import datetime
import wandb
import argparse
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

from pathlib import Path
import sys

# BASE_PATH 설정 (필요에 따라 수정)
BASE_PATH = str(Path(__file__).resolve().parent.parent.parent)  # 예: /Users/yhhan/git/link_dl
print(BASE_PATH, "!!!!!")
sys.path.append(BASE_PATH)

# 데이터셋 클래스 정의
class TitanicDataset(Dataset):
    def __init__(self, csv_file, is_test=False):
        self.data = pd.read_csv(csv_file)

        if is_test:
            self.passenger_ids = self.data['PassengerId']
            # 테스트 데이터셋에서는 'Survived' 열이 없으므로 제거하지 않습니다.
            columns_to_drop = ['PassengerId', 'Name', 'Ticket', 'Cabin', 'Embarked']
            self.data = self.data.drop(columns=columns_to_drop, axis=1, errors='ignore')
        else:
            # 훈련 데이터셋에서는 'Survived' 열이 필요하므로 제거하지 않습니다.
            columns_to_drop = ['PassengerId', 'Name', 'Ticket', 'Cabin', 'Embarked']
            self.data = self.data.drop(columns=columns_to_drop, axis=1, errors='ignore')

        # 'Sex' 인코딩: male=0, female=1
        self.data['Sex'] = self.data['Sex'].map({'male': 0, 'female': 1})

        # 'Age' 결측값을 평균으로 채움
        self.data['Age'] = self.data['Age'].fillna(self.data['Age'].mean())

        if not is_test:
            # 입력 특징과 타겟 분리
            self.X = self.data[['Pclass', 'Sex', 'Age']].values.astype(float)
            self.y = self.data['Survived'].values.astype(float)
        else:
            # 테스트 데이터셋에서는 타겟이 없음
            self.X = self.data[['Pclass', 'Sex', 'Age']].values.astype(float)
            self.y = None

        # 정규화
        self.scaler = StandardScaler()
        self.X = self.scaler.fit_transform(self.X)

        # 텐서로 변환
        self.X = torch.tensor(self.X, dtype=torch.float32)
        if not is_test:
            self.y = torch.tensor(self.y, dtype=torch.float32).unsqueeze(1)  # (N, 1)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        if self.y is not None:
            return self.X[idx], self.y[idx]
        else:
            # 테스트 데이터셋에서는 PassengerId와 함께 반환
            return self.X[idx], self.passenger_ids.iloc[idx]

def get_data(csv_path):
    dataset = TitanicDataset(csv_file=csv_path, is_test=False)
    print(dataset)

    train_size = int(0.8 * len(dataset))
    val_size = len(dataset) - train_size
    train_dataset, validation_dataset = random_split(dataset, [train_size, val_size])
    print(f"Train size: {len(train_dataset)}, Validation size: {len(validation_dataset)}")

    train_data_loader = DataLoader(dataset=train_dataset, batch_size=wandb.config.batch_size, shuffle=True)
    validation_data_loader = DataLoader(dataset=validation_dataset, batch_size=len(validation_dataset))

    return train_data_loader, validation_data_loader

def get_test_data(csv_path):
    test_dataset = TitanicDataset(csv_file=csv_path, is_test=True)
    test_data_loader = DataLoader(dataset=test_dataset, batch_size=len(test_dataset))
    return test_data_loader

class MyModel(nn.Module):
    def __init__(self, n_input, n_output):
        super().__init__()

        self.model = nn.Sequential(
            nn.Linear(n_input, wandb.config.n_hidden_unit_list[0]),
            nn.ELU(),  # ReLU에서 ELU로 변경
            nn.Linear(wandb.config.n_hidden_unit_list[0], wandb.config.n_hidden_unit_list[1]),
            nn.ELU(),  # ReLU에서 ELU로 변경
            nn.Linear(wandb.config.n_hidden_unit_list[1], n_output),
            nn.Sigmoid()  # 이진 분류를 위해 Sigmoid 활성화 추가
        )

    def forward(self, x):
        return self.model(x)

def get_model_and_optimizer():
    my_model = MyModel(n_input=3, n_output=1)  # 입력 특징 수에 맞게 조정
    optimizer = optim.Adam(my_model.parameters(), lr=wandb.config.learning_rate)  # Adam 옵티마이저 사용

    return my_model, optimizer

def training_loop(model, optimizer, train_data_loader, validation_data_loader):
    n_epochs = wandb.config.epochs
    loss_fn = nn.BCELoss()  # 이진 분류를 위한 손실 함수
    next_print_epoch = 100

    for epoch in range(1, n_epochs + 1):
        model.train()
        loss_train = 0.0
        all_preds_train = []
        all_targets_train = []

        for train_batch in train_data_loader:
            input, target = train_batch
            output_train = model(input)
            loss = loss_fn(output_train, target)
            loss_train += loss.item()

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            preds = (output_train > 0.5).float()
            all_preds_train.extend(preds.cpu().numpy())
            all_targets_train.extend(target.cpu().numpy())

        train_loss = loss_train / len(train_data_loader)
        train_accuracy = accuracy_score(all_targets_train, all_preds_train)
        train_precision = precision_score(all_targets_train, all_preds_train)
        train_recall = recall_score(all_targets_train, all_preds_train)
        train_f1 = f1_score(all_targets_train, all_preds_train)

        model.eval()
        loss_validation = 0.0
        all_preds_val = []
        all_targets_val = []

        with torch.no_grad():
            for validation_batch in validation_data_loader:
                input, target = validation_batch
                output_validation = model(input)
                loss = loss_fn(output_validation, target)
                loss_validation += loss.item()

                preds = (output_validation > 0.5).float()
                all_preds_val.extend(preds.cpu().numpy())
                all_targets_val.extend(target.cpu().numpy())

        val_loss = loss_validation / len(validation_data_loader)
        val_accuracy = accuracy_score(all_targets_val, all_preds_val)
        val_precision = precision_score(all_targets_val, all_preds_val)
        val_recall = recall_score(all_targets_val, all_preds_val)
        val_f1 = f1_score(all_targets_val, all_preds_val)

        wandb.log({
            "Epoch": epoch,
            "Training Loss": train_loss,
            "Validation Loss": val_loss,
            "Training Accuracy": train_accuracy,
            "Validation Accuracy": val_accuracy,
            "Training Precision": train_precision,
            "Validation Precision": val_precision,
            "Training Recall": train_recall,
            "Validation Recall": val_recall,
            "Training F1 Score": train_f1,
            "Validation F1 Score": val_f1
        })

        if epoch >= next_print_epoch:
            print(
                f"Epoch {epoch}, "
                f"Training Loss: {train_loss:.4f}, "
                f"Validation Loss: {val_loss:.4f}, "
                f"Training Acc: {train_accuracy:.4f}, "
                f"Validation Acc: {val_accuracy:.4f}, "
                f"Training Precision: {train_precision:.4f}, "
                f"Validation Precision: {val_precision:.4f}, "
                f"Training Recall: {train_recall:.4f}, "
                f"Validation Recall: {val_recall:.4f}, "
                f"Training F1: {train_f1:.4f}, "
                f"Validation F1: {val_f1:.4f}"
            )
            next_print_epoch += 100

def predict_test(model, test_data_loader, output_path):
    model.eval()
    all_preds = []
    all_passenger_ids = []

    with torch.no_grad():
        for test_batch in test_data_loader:
            input, passenger_ids = test_batch
            output = model(input)
            preds = (output > 0.5).int()
            # preds를 1차원으로 변환하여 리스트에 추가
            all_preds.extend(preds.cpu().numpy().flatten())
            all_passenger_ids.extend(passenger_ids.cpu().numpy())

    # 결과를 CSV 파일로 저장
    preds_df = pd.DataFrame({
        'PassengerId': all_passenger_ids,
        'Survived': all_preds
    })
    preds_df.to_csv(output_path, index=False)
    print(f"Test predictions saved to {output_path}")


def main(args):
    current_time_str = datetime.now().astimezone().strftime('%Y-%m-%d_%H-%M-%S')

    config = {
        'epochs': args.epochs,
        'batch_size': args.batch_size,
        'learning_rate': args.learning_rate,
        'n_hidden_unit_list': args.hidden_units,  # 리스트로 받도록 수정
    }

    wandb.init(
        mode="online" if args.wandb else "disabled",
        project="titanic_model_training",
        notes="Titanic survival prediction experiment",
        tags=["titanic", "binary_classification"],
        name=current_time_str,
        config=config
    )
    print(args)
    print(wandb.config)

    # 훈련 및 검증 데이터 로더
    train_data_loader, validation_data_loader = get_data(csv_path=args.train_csv_path)

    # 모델 및 옵티마이저 초기화
    model, optimizer = get_model_and_optimizer()

    # 훈련 루프 실행
    training_loop(
        model=model,
        optimizer=optimizer,
        train_data_loader=train_data_loader,
        validation_data_loader=validation_data_loader
    )

    # 테스트 데이터 로더 (타겟 없음)
    test_data_loader = get_test_data(csv_path=args.test_csv_path)

    # 테스트 데이터에 대한 예측 수행 및 저장
    predict_test(model, test_data_loader, output_path=args.output_path)

    wandb.finish()

if __name__ == "__main__":
    parser = argparse.ArgumentParser()

    parser.add_argument(
        "--wandb", action=argparse.BooleanOptionalAction, default=False, help="Enable or disable WandB logging"
    )

    parser.add_argument(
        "-b", "--batch_size", type=int, default=32, help="Batch size (int, default: 32)"
    )

    parser.add_argument(
        "-e", "--epochs", type=int, default=100, help="Number of training epochs (int, default: 100)"
    )

    parser.add_argument(
        "--learning_rate", type=float, default=1e-3, help="Learning rate (float, default: 1e-3)"
    )

    parser.add_argument(
        "--hidden_units", type=int, nargs='+', default=[64, 32], help="List of hidden units (default: [64, 32])"
    )

    # 수정된 인자
    parser.add_argument(
        "--train_csv_path", type=str, required=True, help="Path to the training CSV dataset"
    )

    parser.add_argument(
        "--test_csv_path", type=str, required=True, help="Path to the testing CSV dataset"
    )

    parser.add_argument(
        "--output_path", type=str, required=True, help="Path to save test predictions"
    )

    args = parser.parse_args()

    main(args)


# 요구사항3

[ELU로 선택 사유 배경]    1. 	전체적인 정확도와 균형있는 성능을 우선으로 선택    2. -	Recall을 우선시한다면 ELU로 선택    3. 
-	Validation Accuracy와 F1 Score의 경우를 봤을 때 ReLU가 더 좋은 것으로 판    4. 
-	Tranining Loss의 경우엔 낮을수록 좋지만, 너무 낮아지면 과적합의 우려가     5. .
-	Training Accuracy와 Training Precision은 전체적으로 높을수록    6. 다.
-	Epoch의 경우 300으로 진행함. 400이상부터는 효과적이지 않고 오히려 성능이 떨어지는것을
 [했다.
-	명령어를 통해서 계속해 수정 과정]
    정해나갔다.
-	python f_my_model_training_with_argparse_wandb.py --train_csv_path train.csv --test_csv_path test.csv --output_path test_predictions_ELU.csv --wandb --batch_size 64 --epochs 200 --learning_rate 0.001 --hidden_units     128 64
-	python f_my_model_training_with_argparse_wandb.py --train_csv_path train.csv --test_csv_path test.csv --output_path test_predictions_ELU.csv --wandb --batch_size 64 --epochs 300 --learning_rate 0.001 --hidden_unit
[시도]
s 128 64
-	hiddenunits의 값도 수정했다. 은닉층이 깊을수록 학습을 많이 할 것으로 판단했기 때문이다. 하지만 값을 많이할수록 성능이 좋다라고 말할 수가 없다고 알고있어서 -> 효과를 보진 못했다
[코드 생성 및 submission.csv 생성] -- --output_path의 부분을 submission.csv로 변경하면 됨(.py 기준, 쥬파터의 경우엔 코드내부에서 변경) 256으로 진행.
-	python f_my_model_training_with_argparse_wandb.py --train_csv_path train.csv --test_csv_path test.csv --output_path test_predictions_ELU.csv --wandb --batch_size 64 --epochs 200 --learning_rate 0.001 --hidden_units 256 64


# 요구사항4

![leaderboard_ranking](./leaderboard_ranking.png)

## 발견

- Leaderboard의 경우 마지막 제출을 기준으로 판단하기도 함. 

![competition78](./78Scores_leaderboard2.png)

# 소감

많은 시간을 투자하진 못했지만 그래도 짧은 시간에 최대한 시도해보려고 했던게 epoch를 200~400정도까지 바꿔가며 추세를 판단했습니다.

batch 수는 한번만 하긴했지만 예를 들어 32라든가, learning rate도 0.0005로도 했던 것 같은데 시간이 오래걸리는만큼 세밀하게 학습할거라는 기대를 했지만 왜인지는 몰라도 그렇게 분석 정확도가 높지가 않던데 어떤식으로 해야 올라갈까를 고민하게된 계기가 되었으며 여러가지로 어떻게하면 올릴 수 있을까와. 어떤것을 올리면 어떤것도 영향을 끼칠까도 고민을 하고 시도하고 싶은 계기가 되었습니다. 

여러가지로 본인이 너무 모른다는 생각을 하게 되었지만 유익한 경험이었습니다.

감사합니다. 